# Bird Species Classifier for AML project
## University of Vienna, S2022
---
### Goal: An image recognition model
#### Open Questions:
* Which methods to use? 
* Which model to train?
* What is the class_dict.csv for?
* Do we need both valid set and test set?

In [81]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

# Load data
#from tf.keras.preprocessing.image import ImageDataGenerator

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Models
from sklearn.linear_model import LogisticRegression
#...
# Model Selection
from sklearn.metrics import confusion_matrix, classification_report

# EDA
## Let's get started
The idea of exploratory data analysis is to get more insight into the data by looking at the dataframe.

In [82]:
path = "../input/100-bird-species"
trainpath = "../input/100-bird-species/train"
validpath = "../input/100-bird-species/valid"
testpath = "../input/100-bird-species/test"
pretest_path = "../input/100-bird-species/images to test"

birds_english = path+"/birds.csv"
birds_latin = path+"/birds latin names.csv"

In [83]:
birds_df = pd.read_csv(birds_english)
birds_df.head()

,class index,filepaths,labels,data set
0,0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train
1,0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train
2,0,train/ABBOTTS BABBLER/003.jpg,ABBOTTS BABBLER,train
3,0,train/ABBOTTS BABBLER/004.jpg,ABBOTTS BABBLER,train
4,0,train/ABBOTTS BABBLER/005.jpg,ABBOTTS BABBLER,train


In [84]:
birds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62388 entries, 0 to 62387
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   class index  62388 non-null  int64 
 1   filepaths    62388 non-null  object
 2   labels       62388 non-null  object
 3   data set     62388 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.9+ MB


In [85]:
birds_df.value_counts("data set").head()

data set
train    58388
test      2000
valid     2000
dtype: int64

In [86]:
birds_df.value_counts("class index").head()

class index
224    259
144    243
287    243
363    227
396    224
dtype: int64

In [87]:
#mask = birds_df['labels'].str.contains("ABBOTTS BABBLER") # Search for text fragment
#mask = birds_df.query('labels == "ABBOTTS BABBLER"') # query for name (case sensitive!)
mask = birds_df.loc[birds_df['class index'] == 62]

print(mask.value_counts("data set"))
mask

data set
train    120
test       5
valid      5
dtype: int64


,class index,filepaths,labels,data set
9072,62,train/BLACK SKIMMER/001.jpg,BLACK SKIMMER,train
9073,62,train/BLACK SKIMMER/002.jpg,BLACK SKIMMER,train
9074,62,train/BLACK SKIMMER/003.jpg,BLACK SKIMMER,train
9075,62,train/BLACK SKIMMER/004.jpg,BLACK SKIMMER,train
9076,62,train/BLACK SKIMMER/005.jpg,BLACK SKIMMER,train
...,...,...,...,...
60698,62,valid/BLACK SKIMMER/1.jpg,BLACK SKIMMER,valid
60699,62,valid/BLACK SKIMMER/2.jpg,BLACK SKIMMER,valid
60700,62,valid/BLACK SKIMMER/3.jpg,BLACK SKIMMER,valid
60701,62,valid/BLACK SKIMMER/4.jpg,BLACK SKIMMER,valid


# Prepare data for model

In [88]:
# Read in images from folders (train, valid, test)


#### Define train, validation and test data X and y

In [89]:
X_train = birds_df.loc[birds_df['data set']=="train"].drop("class index", axis=1).copy()
y_train = birds_df.loc[birds_df['data set']=="train"]["class index"].copy()

X_valid = birds_df.loc[birds_df['data set']=="valid"].drop("class index", axis=1).copy()
y_valid = birds_df.loc[birds_df['data set']=="valid"]["class index"].copy()

X_test = birds_df.loc[birds_df['data set']=="test"].drop("class index", axis=1).copy()
y_test = birds_df.loc[birds_df['data set']=="test"]["class index"].copy()

---
# README
after the first meeting

### Aufteilung
- keras tensorflow - Clemens
- pytorch - Jakob
- PCA + preprocessing - Lena 


### Methoden 
- pca?
- image segementation
- Wie laden wir die Bilder von der CSV ins Notebook?
- Wieviele Datenreihen brauchen wir? 
data set
| train    58388 | test      2000 | valid     2000 |
